# Video replication with Amazon Nova

Imagine you want to replicate a captivating scene from your favorite movie or create a video that evokes the same powerful emotion. In this notebook, we will guide you through the process of bringing your vision to life using **Amazon Nova Canvas** and **Amazon Nova Reel** on **Amazon Bedrock**.

Let's get started!

In [ ]:
# Install libraries
%pip install moviepy --quiet
%pip install --upgrade boto3 --quiet
%pip install --upgrade ipython opencv-python-headless --quiet

In [ ]:
# Import libraries
import os
import sys
import json
import boto3
import base64
import logging
from datetime import datetime
from IPython.display import display, Image, Video, IFrame

# Local video util library
sys.path.append('../../video-generation/python')
import amazon_video_util

In [ ]:
# Constants
IMAGE_MODEL_ID = "amazon.nova-canvas-v1:0"
VIDEO_MODEL_ID = "amazon.nova-reel-v1:0"
# Selected an existing Amazon S3 bucket
VIDEO_TARGET_S3_BUCKET = "bedrock-video-generation-us-east-1-ew4lh9"

In [ ]:
# Create the logger
logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
logger = logging.getLogger(__name__)

# Set default region and credentials
boto3.setup_default_session(
    region_name="us-east-1"
)

# Create the Amazon Bedrock Runtime client
client = boto3.client("bedrock-runtime")

If you were born in the 1980s, this is undoubtedly an intro you would remember!

In [ ]:
display(IFrame("https://www.youtube.com/embed/VzZN9AVBS1I?start=89", width=512, height=256))

Suppose that you want to create a video that essence the suspense of the previous scene: a helicopter hovering near a lush green island, seemingly poised for an enigmatic purpose. Below is the request we'll submit to **Amazon Nova Reel** to bring this moment to life (review each comment to understand each parameter).

In [ ]:
request = {
    "taskType": "TEXT_VIDEO", # Only supported value currently
    "textToVideoParams": {
        "text": # Describe the camera position and the main object/character
                "Aerial shot of a grey and blue helicopter with blades moving rapidly." +
                # Describe the object/character movement
                "The helicopter is heading an imposing island with lush mountains rising from stormy waters." +
                # Describe any other relevant element
                "A solitary sea stack stands isolated in the bottom left." +
                # Describe light
                "Moody storm clouds cast dramatic shadows across the landscape." +
                # Describe colors
                "Emerald greens contrasted with steel grays and deep blues." +
                # Describe style and quality
                "Cinematic, 4K." +
                # Describe the camera movement
                "Dolly forward."
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,  # Only supported value currently
        "fps": 24,  # Only supported value currently
        "dimension": "1280x720",  # Only supported value currently
        "seed": 1 # You can randomize this for different results
    },
}

Let's invoke **Amazon Nova Reel**.

In [ ]:
try:
    # Start the asynchronous video generation job
    invocation = client.start_async_invoke(
        modelId=VIDEO_MODEL_ID,
        modelInput=request,
        outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{VIDEO_TARGET_S3_BUCKET}"}},
    )

    # Invocation Amazon Resource Name (ARN) to monitor the video generation
    invocation_arn = invocation["invocationArn"]

    # Save the invocation details for monitoring (helpful for debugging and reporting feedback)
    amazon_video_util.save_invocation_info(invocation, VIDEO_MODEL_ID)

except Exception as e:
    logger.error(e)

In [ ]:
# Monitoring and downloading the video
output_directory_video = amazon_video_util.monitor_and_download_video(invocation_arn, "output")

In [ ]:
Video(output_directory_video, embed=True, height=300)

What do you think? As you can see, the prompt utilized here shows a high level of organization and specificity. To enhance such prompts, you can take advantage of **Amazon Nova Pro** to suggest alternative wording, improve descriptive elements, or restructure your prompt entirely to make it more effective.

Additionally [Amazon Bedrock Playground](https://docs.aws.amazon.com/bedrock/latest/userguide/playgrounds.html) can be crucial in the testing part allowing to quickly iterate and copy the API request once the results are satisfying.

Finally, if you want to deep dive into the **Amazon Nova Reel** best practices, be sure to check out this [documentation](https://docs.aws.amazon.com/nova/latest/userguide/prompting-video-generation.html) and this great [blog]( https://aws.amazon.com/blogs/machine-learning/image-and-video-prompt-engineering-for-amazon-nova-canvas-and-amazon-nova-reel/) full of examples.

Now we are using a different approach - starting with a base image and generating dynamic movement from it. This method ensures consistent visual quality while creating compelling motion through precise camera direction.

Below is the request we'll use to generate the source image of the video (review each comment to understand each parameter). We will use the same prompt text as for the video but excluding the helicopter, the movement, and changing the camera position.


In [ ]:
request = {
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": """Low angle shot of an imposing island with lush mountains rising from stormy waters.
                   A solitary sea stack stands isolated in the bottom left.
                   Moody storm clouds cast dramatic shadows across the landscape.
                   Emerald greens contrasted with steel grays and deep blues.
                   Cinematic, 4K."""
    },
    "imageGenerationConfig": {
        "numberOfImages": 1, # Number of images to generate
        # Supported image resolutions: https://docs.aws.amazon.com/nova/latest/userguide/image-gen-access.html#image-gen-resolutions
        "height": 720,
        "width": 1280,
        "quality": "premium", # premium or standard
        "cfgScale": 10, # Image adherence to the prompt (1.1 to 10, lower values more randomness)
        "seed": 45 # You can randomize this for different results
    }
}

Let's invoke **Amazon Nova Canvas**.

In [ ]:
try:
    # Invoke the model
    response = client.invoke_model(
        modelId=IMAGE_MODEL_ID,
        body=json.dumps(request)
    )
    # Parse the response
    response_body = json.loads(response["body"].read())
    
    # Extract the base64 image data
    base64_image = response_body["images"][0]
    
    # Create output directory if it doesn't exist
    output_dir = "output"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Generate timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Create filename with random number
    image_filename = f"image_{timestamp}.png"
    image_path = os.path.join(output_dir, image_filename)
    with open(image_path, "wb") as f:
        f.write(base64.b64decode(base64_image))

except Exception as e:
    logger.error(e)

In [ ]:
display(Image(filename=image_path, width=512))

We use the previous image as input and as prompt text, we only detail the movement of the camera: Dolly forward.

In [ ]:
request = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": "Dolly forward.",
        "images": [
            {
                "format": "png", # png or jpeg
                "source": {
                    "bytes": base64_image
                }
            }
        ]},
    "videoGenerationConfig": {
        "durationSeconds": 6,
        "fps": 24,
        "dimension": "1280x720",
        "seed": 1
    },
}

Let's invoke **Amazon Nova Reel**.

In [ ]:
try:
    # Start the asynchronous video generation job
    invocation = client.start_async_invoke(
        modelId=VIDEO_MODEL_ID,
        modelInput=request,
        outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{VIDEO_TARGET_S3_BUCKET}"}},
    )

    # Invocation Amazon Resource Name (ARN) to monitor the video generation
    invocation_arn = invocation["invocationArn"]

    # Save the invocation details for monitoring (helpful for debugging and reporting feedback)
    amazon_video_util.save_invocation_info(invocation, VIDEO_MODEL_ID)

except Exception as e:
    logger.error(e)

In [ ]:
# Monitoring and downloading the video
output_directory_video = amazon_video_util.monitor_and_download_video(invocation_arn, "output")

In [ ]:
Video(output_directory_video, embed=True, height=300)

Let's continue with another scene. Have you seen this famous dust explosion in slow motion?

In [ ]:
display(IFrame("https://www.youtube.com/embed/o66mUfzI3iw?start=185", width=512, height=256))

**Amazon Nova Reel** not only allows you to control the camera movement, but also the speed. Let's try to replicate the previous epic explosion.

In [ ]:
request = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": """Slow motion massive dust explosion in an urban setting.
                   Dust and debris erupt up to the sky occuping the full image.
                   Cinematic, 4K."""
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,
        "fps": 24,
        "dimension": "1280x720",
        "seed": 0
    },
}

In [ ]:
try:
    # Start the asynchronous video generation job
    invocation = client.start_async_invoke(
        modelId=VIDEO_MODEL_ID,
        modelInput=request,
        outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{VIDEO_TARGET_S3_BUCKET}"}},
    )

    # Invocation Amazon Resource Name (ARN) to monitor the video generation
    invocation_arn = invocation["invocationArn"]

    # Save the invocation details for monitoring (helpful for debugging and reporting feedback)
    amazon_video_util.save_invocation_info(invocation, VIDEO_MODEL_ID)

except Exception as e:
    logger.error(e)

In [ ]:
# Monitoring and downloading the video
output_directory_video = amazon_video_util.monitor_and_download_video(invocation_arn, "output")

In [ ]:
Video(output_directory_video, embed=True, height=300)

Is it close to the real one? If you want to deep dive in the camera positions and speed, please check this [documentation](https://docs.aws.amazon.com/nova/latest/userguide/prompting-video-camera-control.html).

Let's embark on a adventure into the depths of space. I'm inviting you to help recreate one of the most memorable scenes from cinema's greatest space epics. Can you guess which film moment I'm thinking of?

In [ ]:
display(IFrame("https://www.youtube.com/embed/qckDs0ODemg?start=86", width=512, height=256))

In [ ]:
request = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": """First-person view inside a spacecraft cockpit entering hyperspace.
                   Stars visible through windshield begin stretching into blue-white streaks, creating a radial tunnel effect.
                   Control panels with blinking lights frame the view.
                   The cockpit illuminates with cool blue glow as the hyperjump intensifies.
                   Capture the sensation of incredible speed from the safety of the pilot's seat."""
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,
        "fps": 24,
        "dimension": "1280x720",
        "seed": 2
    },
}

In [ ]:
try:
    # Start the asynchronous video generation job
    invocation = client.start_async_invoke(
        modelId=VIDEO_MODEL_ID,
        modelInput=request,
        outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{VIDEO_TARGET_S3_BUCKET}"}},
    )

    # Invocation Amazon Resource Name (ARN) to monitor the video generation
    invocation_arn = invocation["invocationArn"]

    # Save the invocation details for monitoring (helpful for debugging and reporting feedback)
    amazon_video_util.save_invocation_info(invocation, VIDEO_MODEL_ID)

except Exception as e:
    logger.error(e)

In [ ]:
# Monitoring and downloading the video
output_directory_video = amazon_video_util.monitor_and_download_video(invocation_arn, "output")

In [ ]:
Video(output_directory_video, embed=True, height=300)

Feel free to modify this notebook to try to replicate your favorite scenes from your preferred movies!